In [104]:
# just some standard import...
from qiskit import pulse
import os, json
import numpy as np
import matplotlib.pyplot as plt
from qiskit.pulse import Play, Schedule, DriveChannel, ControlChannel, Waveform
from qiskit.pulse.library import drag, GaussianSquare
from qiskit.visualization import SchedStyle
from qiskit import QuantumCircuit, Aer, IBMQ
from qiskit.test.mock import FakeAthens

# account + backend
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science')
backend = provider.get_backend('ibmq_casablanca')

# plot more beautiful
style = SchedStyle(figsize=(20, 10))
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [15, 15]


C:\ProgramData\Anaconda3\envs\walter\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2021-04-08 18:12:36,191: Credentials are already in use. The existing account in the session will be replaced.


In [105]:
backend_config = backend.configuration()
backend_defaults = backend.defaults()
dt=backend_config.dt  # hardware resolution
backend.configuration().parametric_pulses = [] # will allow us to send a larger waveform for our experiments
inst_sched_map = backend_defaults.instruction_schedule_map 
ham_params = backend_config.hamiltonian['vars']

In [106]:
backend_config.n_uchannels # Number of u-channels.

12

In [107]:
backend_config.u_channel_lo  # U-channel relationship on device los.

[[UchannelLO(1, (1+0j))],
 [UchannelLO(0, (1+0j))],
 [UchannelLO(2, (1+0j))],
 [UchannelLO(3, (1+0j))],
 [UchannelLO(1, (1+0j))],
 [UchannelLO(1, (1+0j))],
 [UchannelLO(5, (1+0j))],
 [UchannelLO(5, (1+0j))],
 [UchannelLO(3, (1+0j))],
 [UchannelLO(4, (1+0j))],
 [UchannelLO(6, (1+0j))],
 [UchannelLO(5, (1+0j))]]

In [108]:
backend_config.qubit_lo_range # Qubit lo ranges for each qubit with form (min, max) in GHz.

[[4322079641.29924, 5322079641.29924],
 [4259838438.8134046, 5259838438.813404],
 [4407349586.838664, 5407349586.838664],
 [4378965150.40599, 5378965150.40599],
 [4370907055.445278, 5370907055.445278],
 [4463895473.770882, 5463895473.770882],
 [4677098305.340573, 5677098305.340573]]

In [109]:
backend_config.meas_lo_range #  Measurement lo ranges for each qubit with form (min, max) in GHz.

[[6784054663.0, 7784054663.0],
 [6885438514.0, 7885438514.0],
 [6823180030.0, 7823180030.0],
 [6730756417.0, 7730756417.0],
 [6780310863.0, 7780310863.0],
 [6646993373.0, 7646993373.0],
 [6900652534.0, 7900652534.0]]

In [110]:
backend_config.coupling_map # The coupling map for the device

[[0, 1],
 [1, 0],
 [1, 2],
 [1, 3],
 [2, 1],
 [3, 1],
 [3, 5],
 [4, 5],
 [5, 3],
 [5, 4],
 [5, 6],
 [6, 5]]

In [111]:
backend_config.description 

'7 qubit device'

In [112]:
backend_config.channels

{'acquire0': {'operates': {'qubits': [0]},
  'purpose': 'acquire',
  'type': 'acquire'},
 'acquire1': {'operates': {'qubits': [1]},
  'purpose': 'acquire',
  'type': 'acquire'},
 'acquire2': {'operates': {'qubits': [2]},
  'purpose': 'acquire',
  'type': 'acquire'},
 'acquire3': {'operates': {'qubits': [3]},
  'purpose': 'acquire',
  'type': 'acquire'},
 'acquire4': {'operates': {'qubits': [4]},
  'purpose': 'acquire',
  'type': 'acquire'},
 'acquire5': {'operates': {'qubits': [5]},
  'purpose': 'acquire',
  'type': 'acquire'},
 'acquire6': {'operates': {'qubits': [6]},
  'purpose': 'acquire',
  'type': 'acquire'},
 'd0': {'operates': {'qubits': [0]}, 'purpose': 'drive', 'type': 'drive'},
 'd1': {'operates': {'qubits': [1]}, 'purpose': 'drive', 'type': 'drive'},
 'd2': {'operates': {'qubits': [2]}, 'purpose': 'drive', 'type': 'drive'},
 'd3': {'operates': {'qubits': [3]}, 'purpose': 'drive', 'type': 'drive'},
 'd4': {'operates': {'qubits': [4]}, 'purpose': 'drive', 'type': 'drive'},
 '

In [113]:
backend_config.u_channel_lo  # U-channel relationship on device los.

[[UchannelLO(1, (1+0j))],
 [UchannelLO(0, (1+0j))],
 [UchannelLO(2, (1+0j))],
 [UchannelLO(3, (1+0j))],
 [UchannelLO(1, (1+0j))],
 [UchannelLO(1, (1+0j))],
 [UchannelLO(5, (1+0j))],
 [UchannelLO(5, (1+0j))],
 [UchannelLO(3, (1+0j))],
 [UchannelLO(4, (1+0j))],
 [UchannelLO(6, (1+0j))],
 [UchannelLO(5, (1+0j))]]

In [114]:
def cx_pulse_instructions(qc: int, qt: int) -> Schedule:
    """Retrieve the CNOT pulse schedule for the given
    qubit pair from the backend defaults.
    
    Args:
      qc: control qubit index
      qt: target qubit index
    """
    if [qc, qt] not in backend_config.coupling_map:
        print('Qubit pair has no direct cross resonance!')
    else:
        cx = inst_sched_map.get('cx', qubits=[qc, qt])
        return cx
    
def get_cr_chan_inst(qc: int, qt: int) -> (ControlChannel, Waveform):
    """Retrieve the cross resonance pulse channel and waveform
    for the qubit pair from the cx (CNOT) schedule.
    
    Args:
      qc: control qubit index
      qt: target qubit index
    """
    cx = cx_pulse_instructions(qc, qt)
    idx = 0
    # look for first Play instruction on a ControlChannel
    while (type(cx.instructions[idx][1].channels[0]) is not ControlChannel) or \
        (type(cx.instructions[idx][1]) is not Play):
        idx += 1
    return (cx.instructions[idx][1].channels[0], cx.instructions[idx][1].pulse)

In [115]:
from collections import namedtuple

ct_pair = namedtuple('ct_pair', 
                     ['target', # index number of target qubit
                      'control', # index num of control qubit
                     ])

In [116]:
couples = dict(zip(["c"+str(p[0]) +"t"+ str(p[1])for p in backend_config.coupling_map],
         [ct_pair(p[0],p[1]) for p in backend_config.coupling_map]))

couples

{'c0t1': ct_pair(target=0, control=1),
 'c1t0': ct_pair(target=1, control=0),
 'c1t2': ct_pair(target=1, control=2),
 'c1t3': ct_pair(target=1, control=3),
 'c2t1': ct_pair(target=2, control=1),
 'c3t1': ct_pair(target=3, control=1),
 'c3t5': ct_pair(target=3, control=5),
 'c4t5': ct_pair(target=4, control=5),
 'c5t3': ct_pair(target=5, control=3),
 'c5t4': ct_pair(target=5, control=4),
 'c5t6': ct_pair(target=5, control=6),
 'c6t5': ct_pair(target=6, control=5)}

In [117]:
controlchannel_pairing_dict = dict(zip( 
    ["u"+str(x) for x in range(backend_config.n_uchannels)],
    [ct_pair(backend_config.channels.get("u"+str(x)).get('operates').get('qubits')[0],
             backend_config.channels.get("u"+str(x)).get('operates').get('qubits')[1])
             for x in range(backend_config.n_uchannels)]))

In [118]:
controlchannel_pairing_dict

{'u0': ct_pair(target=0, control=1),
 'u1': ct_pair(target=1, control=0),
 'u2': ct_pair(target=1, control=2),
 'u3': ct_pair(target=1, control=3),
 'u4': ct_pair(target=2, control=1),
 'u5': ct_pair(target=3, control=1),
 'u6': ct_pair(target=3, control=5),
 'u7': ct_pair(target=4, control=5),
 'u8': ct_pair(target=5, control=3),
 'u9': ct_pair(target=5, control=4),
 'u10': ct_pair(target=5, control=6),
 'u11': ct_pair(target=6, control=5)}

In [119]:
uchans, crpulses, cxinstructs, crparams, fake_direction	 = [],[],[],[],[]

for pair in controlchannel_pairing_dict.values():
    uc, crp = get_cr_chan_inst(pair.control, pair.target)
    uchans.append(uc)
    crpulses.append(crp)
    crparams.append(crp.parameters)
    cxins = cx_pulse_instructions(pair.control, pair.target)
    cxinstructs.append(cxins)
    # import part: figure out direction
    fake_direction.append(len(cx_pulse_instructions(pair.control, 
                                                    pair.target))> len(cx_pulse_instructions(pair.target,pair.control)))

In [120]:
for pair in controlchannel_pairing_dict.values():
    if len(cx_pulse_instructions(pair.control, pair.target)) < len(cx_pulse_instructions(pair.target, pair.control)):
        uc, crp = get_cr_chan_inst(pair.control, pair.target)
        print("\n",uc,"irradites \n control qubit", pair.control, 
              "with tone at frequency of target qubit",pair.target,"\n\t with CR pulse",crp)


 ControlChannel(1) irradites 
 control qubit 1 with tone at frequency of target qubit 0 
	 with CR pulse GaussianSquare(duration=880, amp=(0.09365219106688942+0.19751548267987415j), sigma=64, width=624)

 ControlChannel(5) irradites 
 control qubit 3 with tone at frequency of target qubit 1 
	 with CR pulse GaussianSquare(duration=560, amp=(-0.322719233320817-0.1620689803892814j), sigma=64, width=304)

 ControlChannel(2) irradites 
 control qubit 1 with tone at frequency of target qubit 2 
	 with CR pulse GaussianSquare(duration=736, amp=(0.6723040722694984+0.32361397301952594j), sigma=64, width=480)

 ControlChannel(8) irradites 
 control qubit 5 with tone at frequency of target qubit 3 
	 with CR pulse GaussianSquare(duration=736, amp=(0.20178039200998332-0.09406804656678261j), sigma=64, width=480)

 ControlChannel(9) irradites 
 control qubit 5 with tone at frequency of target qubit 4 
	 with CR pulse GaussianSquare(duration=656, amp=(-0.23413215649986446+0.12245186116846353j), sig

In [121]:
# ControlChannel(5) irradites control qubit 3 with tone at frequency of target qubit 1 
############### -> means channel 5 is connected to qubit 3

In [122]:
for pair in controlchannel_pairing_dict.values():
    uc, crp = get_cr_chan_inst(pair.control, pair.target)
    c_t_direction, t_c_direction = cx_pulse_instructions(pair.control, pair.target) , cx_pulse_instructions(pair.target, pair.control)
    print("\n\n\t",uc,pair)
    print("\n\tc_t_direction:\n",c_t_direction)
    print("\n\tt_c_direction: \n", t_c_direction)



	 ControlChannel(1) ct_pair(target=0, control=1)

	c_t_direction:
 Schedule((0, ShiftPhase(1.5707963267948966, DriveChannel(1))), (0, ShiftPhase(1.5707963267948966, ControlChannel(0))), (0, ShiftPhase(1.5707963267948966, ControlChannel(4))), (0, ShiftPhase(1.5707963267948966, ControlChannel(5))), (0, Play(Drag(duration=160, amp=(0.06550345092440805+0.0008497369754255958j), sigma=40, beta=-0.31868983573095205), DriveChannel(0))), (0, Play(Drag(duration=160, amp=(-3.304373512823692e-17-0.1798817158342757j), sigma=40, beta=-0.6347890146472998), DriveChannel(1))), (160, Play(GaussianSquare(duration=880, amp=(0.023929317826418883+0.0005539627235926305j), sigma=64, width=624), DriveChannel(0))), (160, Play(GaussianSquare(duration=880, amp=(0.09365219106688942+0.19751548267987415j), sigma=64, width=624), ControlChannel(1))), (1040, Play(Drag(duration=160, amp=(0.1798817158342757+0j), sigma=40, beta=-0.6347890146472998), DriveChannel(1))), (1200, Play(GaussianSquare(duration=880, amp=(-0.023